<br>
<br>
<font size='6'><u><b>Variable Stars</b></u></font>
<br>

##### Written by Chien-Hsiu Lee, NOAO
##### Revised by Andres Jaramillo

In this activity we are going to look at stars that change their magnitudes (brightness) in time. We can use a certain group of these stars to help estimate distances to other astronomical objects such as M31, our neighboring galaxy.
___

# Table of Contents

* [How to Use This Notebook](#How-to-Use-This-Notebook)
* [Pre-Activity Setup](#Pre-Activity-Setup)
* [Activity 1: Finding the Period of a Star](#Activity-1:-Finding-the-Period-of-a-Star)
    * [Question 1: Can you estimate the period from this diagram?](#Question-1:-Can-you-estimate-the-period-from-this-diagram?)
* [Activity 2: Classify the Variable Star](#Activity-2:-Classify-the-Variable-Star)
    * [Question 2: Using a4/a2 and the period, can you detemine the type of this variable?](#Question-2:-Using-a4/a2-and-the-period,-can-you-detemine-the-type-of-this-variable?)
* [Activity 3: Use Leavitt Law to Estimate Distance](#Activity-3:-Use-Leavitt-Law-to-Estimate-Distance)
    * [Question 3: Can you estimate the distance to the Andromeda galaxy?](#Question-3:-Can-you-estimate-the-distance-to-the-Andromeda-galaxy?)
    * [Bonus Question: Can you estimate the distance to Large Magellanic Cloud?](#Bonus-Question:-Can-you-estimate-the-distance-to-Large-Magellanic-Cloud?)
___

# How to Use This Notebook

The webpage you are in is actually an app - much like the ones on your cellphone. This app consists of cells.

An *input* cell looks like a light grey box with an `In [ ]:` on its left. Input cells each contain code - instructions to make the computer do something.

To activate or select a cell, click anywhere inside of it.

<div class='alert alert-info'>
    <font size='3'><b>Select the cell below and read its contents.</b></font>
</div>

In [ ]:
# Text that follows a "#" is known as a comment.
# Comments do not affect your code in any way.
# You should always read the comments at the top of each cell you interact with.
# Comments will be used to describe what the cell's code is actually doing.

To execute or run a selected cell, hit `[Shift + Enter]` on your keyboard.

<div class='alert alert-info'>
    <font size='3'><b>Select the cell below and read its contents. Then, run the cell.</b></font>
</div>

In [ ]:
# Text that DOESN'T follow a "#" is considered code.
# Lines of code are instructions given to your computer.
# The line of code below is a "print" statement.
# A print statement literally prints out the text between its quotes.

print("Congrats! You have successfully run your first cell!")

Running a cell creates an *output* directly below it. An output can be some text, a graph, an interactive slider, or even nothing at all! For that last case, you know you have run a cell when the `In [ ]:` becomes `In [#]:`, where "#" is any number.

You can learn more about how Jupyter notebooks work at https://try.jupyter.org/
___

# Pre-Activity Setup

In order for any of the activities to work properly, you must import the libraries needed for the code in this notebook.

<div class='alert alert-info'>
    <font size='3'><b>Select and run the cell below.</b></font>
</div>

In [ ]:
#First we need to load some libraries/packages
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii

___

# Activity 1: Finding the Period of a Star

In this exercise, we will learn how to display light curves (brightness as a function of time) and from the light curve, find the period of a variable star.

In [ ]:
#We read in the observation data and plot the light curve
lc=ascii.read('cep1862r.tab')
mjd, mag, err = lc['col1'], lc['col2'], lc['col3']

plt.figure(figsize=(12, 8))
plt.plot(mjd, mag, 'g.')
plt.xlabel('Time [day]', size=20)
plt.ylabel('Brightness [magnitude]', size=20)
plt.title('Light curve of a variable star', size=20)
plt.ylim(20.2,18.8)
#plt.rc('font', weight='bold')
plt.rc('xtick.major', size=10, pad=10)
plt.rc('ytick.major', size=10, pad=10)
plt.rc('ytick', labelsize=20)
plt.rc('xtick', labelsize=20)
plt.show()

This is a good start and there appears to be a periodic nature to the star, so lets find the period.

In [ ]:
#now we use a package to find the period
from gatspy.periodic import LombScargleFast

ls=LombScargleFast()
ls.optimizer.period_range = (0.1,100)
ls.fit(mjd,mag)
period0=ls.best_period
#print("Best period: " + str(period) + " days")

#periodogram
periods = np.linspace(1,100,1000)
scores = ls.score(periods)
plt.figure(figsize=(12, 8))
plt.plot(periods,scores)
plt.xlabel("Period [day]", size=20)
plt.ylabel("Likelihood", size=20)
plt.title("Lomb-Scargle-Periodogram", size=20)
plt.show()


___

## Question 1: Can you estimate the period from this diagram?

That was a great way to make sure we know the period. In fact, if we assume all the data sits on the same periodic function, we should be able to condense all the data to a single period, or "phased light curve". This will make it easier to read and see the period.

From the above diagram of likelihood versus period, you can probably determine the period of the variable star. But for arguments sake, let's play around with different periods, say from 10 to 60 days, and see which of their diagram of period versus brightness make sense.

In [ ]:
#print("Best period: " + str(period) + " days")
# put in a number between 10 and 60
#period=
period = 30.864572200228128

phase = np.zeros(shape=mjd.shape)
phase = np.fmod(mjd/period,1)

plt.figure(figsize=(12, 8))
plt.plot(phase*period,mag,"g.")
plt.plot((phase+1)*period,mag,"r.")
plt.ylim(20.2,18.8)
plt.xlabel('Time [Days]', size=20)
plt.ylabel('Brightness [Magnitude]', size=20)
plt.title('Phased light curve', size=20)
plt.show()

___

# Activity 2: Classify the Variable Star

In this exercise, we will learn how to use a series of cosine and sine functions to model a light curve (so-called Fourier decomposition), and how we can use the resulting coefficients to classify the type of variable star.

In [ ]:
#definition of Fourier decomposition function
from scipy import optimize
pi=3.1415
def FourierDecomp(x, a0, a1, a2,a3,a4, b1, b2, b3, b4):
    return a0 + a1 * np.cos(2*pi* x) + a2 * np.cos(2*pi*2*x) + a3*np.cos(2*pi*3*x) + a4*np.cos(2*pi*4*x) + b1*np.sin(2*pi*x) + b2*np.sin(2*pi*2*x) + b3*np.sin(2*pi*3*x) + b4*np.sin(2*pi*4*x) 

#fit the light curve and show coefficients
params, params_covariance = optimize.curve_fit(FourierDecomp, phase, mag, p0=[19.5, 0, 0, 0, 0, 0, 0, 0, 0])

print(params)

And plot the resulting curve on the data

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(phase*period, mag, "r.", label='Data')
plt.plot((phase+1)*period, mag, "r.", label='Data')
p2=np.linspace(0,1,100)
plt.plot(p2*period, FourierDecomp(p2, params[0], params[1], params[2], params[3], params[4], params[5], params[6], params[7], params[8]), color='black', label='Fitted function')
plt.plot((p2+1)*period,FourierDecomp(p2, params[0], params[1], params[2], params[3], params[4], params[5], params[6], params[7], params[8]), color='black', label='Fitted function')
plt.xlabel('Time [Days]', size=20)
plt.ylabel('Brightness [Magnitude]', size=20)
plt.ylim(20.2,18.8)
plt.show()

From the light curve, use coefficients a4 and a2 to determine a4/a2 vs period.

In [ ]:
print("a4/a2: ",params[4]/params[2], "log period: ", np.log10(period))

___

## Question 2: Using a4/a2 and the period, can you detemine the type of this variable?

To determine the type of variable star from the period, check the classification diagram:

<img src="TeenCafeClass.jpeg">

Now use this period with the Leavitt Law to determine luminosity (i.e., absolute magnitude or intrinsic brightness) and with that, get the distance.
___

# Activity 3: Use Leavitt Law to Estimate Distance

In this exercise, we will fit the data to determine the coefficients of Leavitt law, and from that, we can estimate the distance to our nearest spiral galaxy, M31.

<img src="TeenCafeDistance.jpeg">

In [ ]:
#read in data of Andromeda Cepheids
cep=ascii.read('cep.tab')
per, jmag, hmag, typ = cep['col1'], cep['col2'], cep['col3'], cep['col4']

plt.figure(figsize=(12, 8))
plt.plot(np.log10(per[cep['col4'] == 'FM']), jmag[cep['col4'] == 'FM'], 'm.')
plt.plot(np.log10(per[cep['col4'] == 'FO']), jmag[cep['col4'] == 'FO'], 'y.')

#plt.xscale("log")
plt.xlabel('log(Period) [day]', size=20)
plt.ylabel('Magnitude', size=20)
plt.title('Leavitt Law', size=20)
plt.ylim(22,16)
plt.show()

By fitting a line to the data in the diagram above (showing the Leavitt Law), the absolute magnitude (or luminosity) can be found. With the equation for a line y=ax+b, coefficient "a" is the slope of the line and coefficient "b" is the y value when x is zero. That y value is the absolute magnitude or luminosity of the variable star with a period of one day or log(0)=1. This value will be M1 in Question 3 below.

In [ ]:
def Linear(x, a, b):
    return a*x + b 

params, params_covariance = optimize.curve_fit(Linear,
                                               np.log10(per[cep['col4'] == 'FM']),
                                               jmag[cep['col4'] == 'FM'],
                                               p0=[-1,22])

print("y = a*x + b, a=", params[0], " b=", params[1])

In [ ]:
ppp=np.linspace(1,100,100)
#print p2

plt.figure(figsize=(12, 8))
plt.plot(np.log10(ppp), Linear(np.log10(ppp), params[0], params[1]), color='black', label='Fitted function')

plt.plot(np.log10(per[cep['col4'] == 'FM']), jmag[cep['col4'] == 'FM'], 'm.')
plt.plot(np.log10(per[cep['col4'] == 'FO']), jmag[cep['col4'] == 'FO'], 'y.')

#plt.xscale("log")
plt.xlabel('log(Period) [day]', size=20)
plt.ylabel('Brightness [Magnitude]', size=20)
plt.title('Leavitt Law', size=20)
plt.ylim(23,16)
plt.xlim(0,2)
plt.show()

___

## Question 3: Can you estimate the distance to the Andromeda galaxy?

We can use the distance modulus to convert magnitude to distance:

$M_1 - M_2 = 5 x log_{10}(d_1/d_2)$

or in terms of distance ($d_1$):

$d_1 = d_2 10^{(M_1 - M_2)/5}$

To get d1, what value for M1 do you need to use from the Leavitt Law above?
See previous paragraph before Question 3.

Info you will need for M2 and d2: a 1-day period Cepheid has brightness of -2.1 magnitude at 10 pc.

You can use Python as a calculator to determine the distance. Try it yourself using pow(base,exponent) for base^(exponent). Check your answer with the distance here: https://en.wikipedia.org/wiki/Andromeda_Galaxy

In [ ]:
#Now fill in M1, M2, and d2 below to get d1
M1 = 22.365165657
M2 = -2.1
d2 = 10

d1 = d2 * pow(10,(M1-M2)/5.)

print("Distance to Andromeda =", d1)

___

## Bonus Question: Can you estimate the distance to Large Magellanic Cloud?

In [ ]:
#read in data of LMC Cepheids
cep=ascii.read('lmc.tab')
per, jmag, hmag, typ = cep['col1'], cep['col2'], cep['col3'], cep['col4']

plt.figure(figsize=(12, 8))
plt.plot(np.log10(per[cep['col4'] == 'FM']), jmag[cep['col4'] == 'FM'], 'm.')

#plt.xscale("log")
plt.xlabel('log(Period) [day]', size=20)
plt.ylabel('Magnitude', size=20)
plt.title('Leavitt Law', size=20)
plt.ylim(16.5,9)
plt.xlim(0.0,2)
plt.show()

In [ ]:
params, params_covariance = optimize.curve_fit(Linear, np.log10(per[cep['col4'] == 'FM']), jmag[cep['col4'] == 'FM'], p0=[-1,22])

print("y = a*x + b, a=", params[0], " b=", params[1])
ppp=np.linspace(1,500,100)
#print p2

plt.figure(figsize=(12, 8))
plt.plot(np.log10(ppp), Linear(np.log10(ppp), params[0], params[1]), color='black', label='Fitted function')

plt.plot(np.log10(per[cep['col4'] == 'FM']), jmag[cep['col4'] == 'FM'], 'm.')

#plt.xscale("log")
plt.xlabel('log(Period) [day]', size=20)
plt.ylabel('Brightness [Magnitude]', size=20)
plt.title('Leavitt Law', size=20)
plt.ylim(16.5,9)
plt.xlim(0,2)
plt.show()


In [ ]:
#Now fill in M1, M2, and d2 below to get d1
M1 = 16.4114153606
M2 = -2.1
d2 = 10

d1 = d2 * pow(10,(M1-M2)/5.)

print("Distance to LMC =", d1)

___